In [54]:
import pandas as pd
import datetime
import json
import numpy as np
import string
import math
import re
import matplotlib.pyplot as plt
import seaborn as sns

from torch.utils.data import TensorDataset

import torchtext
from torchtext.vocab import Vectors
from torchtext import data, datasets

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from model import QRDQN

# データのロード

In [55]:
for i, date in enumerate(range(2011, 2019)):
    tmp = pd.read_csv('./data/news/' + str(date) + '.csv', encoding='cp932')
    tmp = tmp[tmp['Company_IDs(TSE)'] == '7203']
    tmp = tmp[['Time_Stamp_Original(JST)', 
                        'Company_Code(TSE)', 
                        'Headline', 
                        'News_Source',
                        'Company_Relevance', 
                        'Keyword_Article']]

    # 欠損除去
    tmp = tmp[~tmp["Keyword_Article"].isnull()]

    # タグ除去
    tmp = tmp[(tmp['News_Source'] == '日経') | 
                        (tmp['News_Source'] == 'ＮＱＮ') |
                        (tmp['News_Source'] == 'ＱＵＩＣＫ') | 
                        (tmp['News_Source'] == 'Ｒ＆Ｉ')]

    tmp.index = pd.to_datetime(tmp["Time_Stamp_Original(JST)"])
    tmp = tmp.drop("Time_Stamp_Original(JST)", axis=1)
    
    if i == 0:
        df1 = tmp.copy()
    else:
        df1 = pd.concat([df1, tmp])

# インデックスを設定

In [56]:
def norm_time(x):
    if x.hour > 15:
        return x + datetime.timedelta(days=1)
    return x

time = pd.to_datetime(df1.index.values)
df1.index = df1.index.map(norm_time)
df1.index = df1.index.date

# 株価を挿入する

In [57]:
# 株価を取り出す
df2 = pd.read_csv('./data/stock_price/7203.csv', index_col=0)
df2.index = pd.to_datetime(df2['date'])
df2.index = df2.index.date
df2 = df2.drop(['date'], axis=1)
df2.head(10)

,adj_close
2011-01-04,3265.0
2011-01-05,3295.0
2011-01-06,3380.0
2011-01-07,3455.0
2011-01-11,3455.0
2011-01-12,3500.0
2011-01-13,3535.0
2011-01-14,3550.0
2011-01-17,3500.0
2011-01-18,3510.0


# 時系列をくっつける

In [58]:
df3 = pd.concat([df1,df2], axis=1, join_axes=[df1.index])
df3['price'] = np.round(df2.pct_change().shift(-1) * 100, 10)
df3['Keyword_Article'] = \
    df3.groupby(level=0).apply(lambda x: ':<pad>:'.join(list(x['Keyword_Article'])))
df3 = df3.dropna()

df3 = df3[~df3.duplicated(subset=['Keyword_Article'])]

/home/ts-zemi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  """Entry point for launching an IPython kernel.


In [59]:
df3.head()

,Company_Code(TSE),Headline,News_Source,Company_Relevance,Keyword_Article,adj_close,price
2011-01-04,7203.0,<日経>◇次世代車の研究開発 名大に国内最大拠点,日経,38,安全:環境:負荷:開発:目指す:開所式:研究拠点:効率:簡素化:次世代:電気自動車:電気:幅...,3265.0,0.918836
2011-01-05,7203.0,<日経>◇12月の中国新車販売、トヨタが単月で過去最高,日経,100,北京:中国:１２月:新車販売台数:前年同月比:増:過去最高:制限:受け:全国:各地:乗用車:...,3295.0,2.579666
2011-01-06,7203.0,<NQN>◇トヨタ社長「今年は後半に晴れ間」 為替は１ドル＝90円を期待,ＮＱＮ,100,豊田:見通し:販売:エコカー補助金:安定的:伸び:株価:為替:水準:日経平均株価:最低:ライ...,3380.0,2.218935
2011-01-07,7203.0,<日経>◇福岡県、自動車の技術者育成へ新組織 年内、中小向け,日経,37,自動車産業:強化:福岡:先端:設置:方針:技術:調査:ニーズ:カリキュラム:大学:受け:生産...,3455.0,0.000000
2011-01-11,7203.0,<日経>◇トヨタ、米ミシガン州に安全研究センター新設,日経,100,先進:安全:子供:高齢者:事故:向上:目指す:米国:大規模:リコール:回収:問題:開催:豊田...,3455.0,1.302460


# csvファイルに保存する

In [60]:
train_date = 2015
test_date = 2016

In [61]:
df4 = pd.concat([df3[['Keyword_Article', 'price']].rename(
                                      columns={'Keyword_Article': 'state', 'price': 'reward'}),
                               df3[['Keyword_Article']].shift(-1).rename(
                                      columns={'Keyword_Article': 'next_state'})], axis=1).dropna()
df4 = df4[['state', 'next_state', 'reward']]

date_year = df4.index.map(lambda x: x.year)

In [62]:
df4[date_year <= train_date].to_csv(
        './data/news/text_train.tsv',
        header=None,
        index=None,
        sep='\t')

In [63]:
df4[(train_date < date_year) & (date_year < test_date)].to_csv(
        './data/news/text_val.tsv',
        header=None,
        index=None,
        sep='\t')

In [64]:
df4[test_date <= date_year].to_csv(
        './data/news/text_test.tsv',
        header=None,
        index=None,
        sep='\t')

# Dataの作成

In [65]:
# 前処理
def preprocessing_text(text):
    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ",") or (p == ":") or (p == "<")or (p == ">"):
            continue
        else:
            text = text.replace(p, " ")

    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    text = re.sub(r'[0-9 ０-９]', '0', text)
    
    return text

# 分かち書き（今回はデータが英語で、簡易的にスペースで区切る）
def tokenizer_punctuation(text):
    return text.strip().split(':')

# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret

In [66]:
max_length = 1000
batch_size = 32

# 読み込んだ内容に対して行う処理を定義
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, 
                            use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, 
                            init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False, dtype=torch.float)

In [67]:
train_ds = torchtext.data.TabularDataset.splits(
    path='./data/news', train='text_train.tsv',
    format='tsv',
    fields=[('Text1', TEXT), ('Text2', TEXT), ('Label', LABEL)])
train_ds = train_ds[0]

# japanese_fasttext_vectors = Vectors(name='./data/news/cc.ja.300.vec')
TEXT.build_vocab(train_ds, 
#                  vectors=japanese_fasttext_vectors,
                 min_freq=10)
TEXT.vocab.freqs

train_dl = torchtext.data.Iterator(
    train_ds, batch_size=batch_size, train=True)

In [68]:
batch = next(iter(train_dl))
print(batch.Text1)
print(batch.Text2)
print(batch.Label)

(tensor([[   2,  738, 1156,  ...,    1,    1,    1],
        [   2,  480,  311,  ...,    1,    1,    1],
        [   2, 1047,   13,  ...,    1,    1,    1],
        ...,
        [   2,   28,  144,  ...,    1,    1,    1],
        [   2,   17,   47,  ...,    1,    1,    1],
        [   2,  137,  498,  ...,    1,    1,    1]]), tensor([137, 214, 147, 597,  45,  22,  19, 142,  18, 134, 328,  76,  27, 258,
         16,  71, 117, 173,  36, 115,  79, 158,  78,  70, 102,  62,  91, 165,
        142,  87,  79,  77]))
(tensor([[   2,  343,  225,  ...,    1,    1,    1],
        [   2,  263,  111,  ...,    1,    1,    1],
        [   2,  631,   21,  ...,    1,    1,    1],
        ...,
        [   2, 1674,  149,  ...,    1,    1,    1],
        [   2,   17,  179,  ...,    1,    1,    1],
        [   2,   28,  108,  ...,    1,    1,    1]]), tensor([200, 258,  31,  35, 102,  61,  21,  80, 138,  96, 128,  96,  95,  45,
        128,  33, 178,  55, 162,  47, 164,  58, 106, 223, 121,  11,  43, 110,
  

In [69]:
batch.Text1[0][1]

tensor([   2,  480,  311,  595,    4,  638,  687,    0,   13,  130,   40,  222,
         101,  860,  970,  111,    7,    7,  113,   31,  309, 1266,  357,    0,
         151,   49,   83,  946, 1304, 2166,    4,   17,  572,   96,   64,    8,
          79,   47,   70,   88,   66,  273,    4,  121,    6,  179,    0,  879,
         154,   51,    0, 1067,   60,  292,   43, 1012,    0,  333,  991,   22,
         352,  481,    0, 2098,    9,    4,   67, 1071,  444,   56,   26, 1034,
         251,   75,   27,  138,    4,  509, 1779,  558,  146,  368,   19,   62,
         204,   20,   16,    4,  830, 1848,    4,    0, 1555,  188, 1095, 1149,
         610, 1548,  306,  120,  479,  227,  579,  682, 1935,  126, 1661,   12,
         888,  322, 1904,  241,  208,   58, 1092,    0, 1247,   11,   92,  553,
        1127, 1482,  375,  605,  898,   48,  134,  936, 1224,  299,  650, 1436,
        1279,  377,   39,   89,  107,  129, 1557,  272,   46,   99,  535,  223,
         645,  217, 1797,   42,   28,  1

# モデル構築

In [51]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [52]:
VOCAB_SIZE = len(TEXT.vocab.freqs)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
PAD_IDX = 1
NUM_QUANTILE = 51
GAMMA = 0.99
cumulative_density = torch.FloatTensor(
            (2 * np.arange(NUM_QUANTILE) + 1) / (2.0 * NUM_QUANTILE)).to(device)

quantile_weight = 1.0 / NUM_QUANTILE

In [53]:
model = QRDQN(TEXT.vocab.vectors, VOCAB_SIZE, EMBEDDING_DIM, N_FILTERS,
                        FILTER_SIZES, PAD_IDX)

target_model = QRDQN(TEXT.vocab.vectors, VOCAB_SIZE, EMBEDDING_DIM, N_FILTERS,
                        FILTER_SIZES, PAD_IDX)

model = model.to(device)
target_model = target_model.to(device)

target_model.load_state_dict(model.state_dict())

<All keys matched successfully>

# 最適化

In [20]:
# 最適化手法
learning_rate = 2.5e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
def accuracy(scores, y):    
    correct = (scores == y)
    acc = correct.sum() / len(correct)
    return acc

def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()
    return acc

def huber(x):
        cond = (x.abs() < 1.0).float().detach()
        return 0.5 * x.pow(2) * cond + (x.abs() - 0.5) * (1.0 - cond)

In [22]:
## テスト
batch = next(iter(train_dl))

In [23]:
num_epochs = 100
TARGET_UPDATE_FREQ = 10
# dataloaders_dict = {'train': train_dl, 'val':val_dl}
dataloaders_dict = {'train': train_dl}

print('----start----')

torch.backends.cudnn.benchmark = True

for epoch in range(num_epochs):
    epi_rewards = []
    neutrals = []
    buys = []
    
    # update target_model
    if epoch % TARGET_UPDATE_FREQ == 0:
        target_model.load_state_dict(model.state_dict())
    
    for batch in (dataloaders_dict['train']):      
        # curr_q
        states = batch.Text1[0].to(device)
        next_states = batch.Text2[0].to(device)
        rewards = batch.Label.to(device)
    
        with torch.no_grad():
            dist = model(states) * quantile_weight
            actions = dist.sum(dim=2).max(1)[1]

            actions = torch.where(torch.randn(len(states)).to(device) >= 0, 
                                                   actions, 
                                                  (actions + 1) % 2)

            selected_actions = actions.detach().cpu().numpy()
            actions = actions.view(states.size(0), 1, 1).expand(-1, -1, NUM_QUANTILE)


        epi_rewards.append((selected_actions * rewards.detach().cpu().numpy()).sum())
        neutrals.append(len(selected_actions[selected_actions == 0]))
        buys.append(len(selected_actions[selected_actions == 1]))
        
        curr_q = model(states).gather(1, actions).squeeze(dim=1)

        # target_q
        with torch.no_grad():

            next_dist = model(next_states) * quantile_weight
            next_action = next_dist.sum(dim=2).max(1)[1].view(next_states.size(0), 1, 1).expand(
                -1, -1, NUM_QUANTILE)

            next_q = target_model(next_states).gather(1, next_action).squeeze(dim=1)
            target_q = rewards.view(-1, 1) + (GAMMA * next_q)

        diff = target_q.t().unsqueeze(-1) - curr_q.unsqueeze(0)
        loss = huber(diff) * torch.abs(
                        cumulative_density.view(1, -1) - (diff < 0).to(torch.float))
        loss = loss.transpose(0, 1)
        loss = loss.mean(1).sum(-1).mean()

        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        for param in model.parameters():
            param.grad.data.clamp_(-1, 1)
        optimizer.step()
    
    print('--------------------')
    print('epoch:', epoch)
    print('loss:', loss.item())
    print('epi_reward:', sum(epi_rewards))
    print('neutrals:', sum(neutrals), '  buys:', sum(buys))

----start----
--------------------
epoch: 0
loss: 5.317849159240723
epi_reward: 0.3729999929200858
neutrals: 513   buys: 506
--------------------
epoch: 1
loss: 4.558788299560547
epi_reward: 0.7419999967096373
neutrals: 515   buys: 504
--------------------
epoch: 2
loss: 4.657007217407227
epi_reward: 0.07799997890833765
neutrals: 500   buys: 519
--------------------
epoch: 3
loss: 4.888540267944336
epi_reward: 0.1149999905610457
neutrals: 488   buys: 531
--------------------
epoch: 4
loss: 4.71846342086792
epi_reward: 0.2929999908665195
neutrals: 480   buys: 539
--------------------
epoch: 5
loss: 5.1030168533325195
epi_reward: 0.5670000109821558
neutrals: 488   buys: 531
--------------------
epoch: 6
loss: 5.282257556915283
epi_reward: 0.6059999867575243
neutrals: 500   buys: 519


KeyboardInterrupt: 

# 描画

In [ ]:
batch = next(iter(train_dl))
states = batch.Text1[0].to(device)
next_states = batch.Text2[0].to(device)
rewards = batch.Label.to(device)

In [ ]:
state = states[8]
actions = (model(state.unsqueeze(0)) * quantile_weight)
dist_action = actions[0].cpu().detach().numpy()
# sns.distplot(dist_action[0], bins=51, color='red')
sns.distplot(dist_action[1], bins=10, color='blue')
plt.show()

In [ ]:
state = states[8]
actions = (model(state.unsqueeze(0)) * quantile_weight)
dist_action = actions[0].cpu().detach().numpy()
# sns.distplot(dist_action[0], bins=51, color='red')
sns.distplot(dist_action[1], bins=30, color='blue')
plt.show()

In [ ]:
import numpy as np
curr_q = np.arange(102).reshape(1, 2, 51)
target_q = np.arange(102).reshape(51, 2, 1)

In [ ]:
(target_q - curr_q).shape

In [ ]:
curr_q